# Using Naive Bayes algorithm for spam detection

In this assigment, you will predict if a sms message is 'spam' or 'ham' (i.e. not 'spam') using the Bernoulli Naive Bayes *classifier*.

The training data is from the UCI Machine Learning Repository: https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection.  Please download the zip file before running the code below. 


## Step 1:  Getting, understanding, and cleaning the dataset


###  Importing the dataset

In [2]:
# Import the usual libraries
import numpy as np 
import pandas as pd  # To read in the dataset we will use the Panda's library
df = pd.read_table('SMSSpamCollection', sep = '\t', header=None, names=['label', 'sms_message'])

# Next we observe the first 5 rows of the data to ensure everything was read correctly
df.head() 

/Users/sunidhibrajesh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: read_table is deprecated, use read_csv instead.
  after removing the cwd from sys.path.


,label,sms_message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


### Data preprocesssing
It would be more convenient if the labels were binary instead of 'ham' and 'spam'.  This way our code can always work with numerical values instead of strings.

In [3]:
df['label']=df.label.map({'spam':1, 'ham':0})
df.head() # Again, lets observe the first 5 rows to make sure everything worked before we continue

,label,sms_message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


### Splitting the dcoument into training set and test set

In [4]:
# This time we will use sklearn's method for seperating the data
from sklearn.model_selection import train_test_split

df_train_msgs, df_test_msgs, df_ytrain, df_ytest = train_test_split(df['sms_message'],df['label'], random_state=0)

#Looking at the train/test split
print("The number of training examples: ", df_train_msgs.shape[0])
print("The number of test exampels: ", df_test_msgs.shape)

print("The first four labels")
print(df_ytrain[0:4])

print("The first four sms messages")
print(df_train_msgs[0:4])


#remove
print("first test message:",df_test_msgs[0:1])

print("first test message label:",df_ytest[0:50])


The number of training examples:  4179
The number of test exampels:  (1393,)
The first four labels
872     0
831     1
1273    0
3314    0
Name: label, dtype: int64
The first four sms messages
872     Its going good...no problem..but still need li...
831     U have a secret admirer. REVEAL who thinks U R...
1273                                                Ok...
3314    Huh... Hyde park not in mel ah, opps, got conf...
Name: sms_message, dtype: object
first test message: 4456    Storming msg: Wen u lift d phne, u say "HELLO"...
Name: sms_message, dtype: object
first test message label: 4456    0
690     1
944     0
3768    0
1189    0
4437    0
3587    1
1982    0
2038    0
2078    0
4224    0
2425    0
3975    0
2690    0
1999    0
3348    0
134     1
3849    0
452     0
4379    0
5216    0
39      0
861     0
467     0
15      1
1362    0
348     0
3171    0
1892    0
2961    0
3487    0
3499    0
3155    1
2433    0
3753    0
489     0
4579    1
5253    0
1755    0
2311    1
2793 

###  Creating the feature vector from the text (feature extraction)

Each message will have its own feature vector.  For each message we will create its feature vector as we discussed in class; we will have a feature for every word in our vocabulary.  The $j$th feature is set to one ($x_j=1$) if the $j$th word from our vocabulary occurs in the message, and set the $j$ feature is set to $0$ otherwise ($x_j=0$).

We will use the sklearn method CountVectorize to create the feature vectors for every messge.

We could have written the code to do this directly by performing the following steps:
* remove capitalization
* remove punctuation 
* tokenize (i.e. split the document into individual words)
* count frequencies of each token 
* remove 'stop words' (these are words that will not help us predict since they occur in most documents, e.g. 'a', 'and', 'the', 'him', 'is' ...

In [5]:
# importing the library
from sklearn.feature_extraction.text import CountVectorizer
# creating an instance of CountVectorizer
# Note there are issues with the way CountVectorizer removes stop words.  To learn more: https://scikit-learn.org/stable/modules/feature_extraction.html#stop-words
vectorizer = CountVectorizer(binary = True, stop_words='english')

# If we wanted to perform Multnomial Naive Bayes, we would include the word counts and use the following code
#vectorizer = CountVectorizer(stop_words='english')

# To see the 'count_vector' object
print(vectorizer)

CountVectorizer(analyzer='word', binary=True, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words='english',
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)


In [6]:
# To see the 'stop words' 
print(vectorizer.get_stop_words())

frozenset({'also', 'few', 'bottom', 'anything', 'front', 'ie', 'namely', 'therefore', 'part', 'enough', 'when', 'moreover', 'against', 'same', 'de', 'whereby', 'be', 'or', 'am', 'call', 'are', 'thus', 'seems', 'together', 'upon', 'such', 'can', 'five', 'cannot', 'someone', 'nevertheless', 'no', 'what', 'do', 'herself', 'inc', 'in', 'hereby', 'somehow', 'thin', 'these', 'elsewhere', 'everywhere', 'has', 're', 'around', 'except', 'anyway', 'than', 'could', 'therein', 'afterwards', 'become', 'any', 'that', 'none', 'four', 'thru', 'toward', 'latter', 'nowhere', 'meanwhile', 'see', 'eight', 'wherein', 'must', 'sometimes', 'should', 'along', 'further', 'wherever', 'neither', 'ourselves', 'next', 'though', 'rather', 'everything', 'under', 'which', 'couldnt', 'will', 'during', 'him', 'there', 'else', 'name', 'have', 'beside', 'is', 'ltd', 'where', 'both', 'for', 'over', 'still', 'several', 'the', 'above', 'otherwise', 'fifty', 'becomes', 'after', 'less', 'their', 'always', 'hereupon', 'sometim

In [7]:
# Create the vocabulary for our feature transformation
vectorizer.fit(df_train_msgs)

# Next we create the feature vectors for both the training data and the test data
X_train = vectorizer.transform(df_train_msgs).toarray() # code to turn the training emails into a feature vector
X_test = vectorizer.transform(df_test_msgs).toarray() # code to turn the test email into a feature vector

# Changing the target vectors data type  
y_train=df_ytrain.to_numpy() # Convereting from a Panda series to a numpy array
y_test = df_ytest.to_numpy()

# To observe what the data looks like 
print("The label of the first training example: ", y_train[0])
print("The first training example: ", X_train[0].tolist())# I needed to covernt the datatype to list so all the feature values would be printed

The label of the first training example:  0
The first training example:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

# Your code goes here

In [17]:
# You should NOT use the features of sklearn library in your code.
#### TO-DO #####
#print our vocabulary 
#print("vocabulary:",vectorizer.get_feature_names())

#getting number of words in one mail
#print(vectorizer.vocabulary_)

np.seterr(divide='ignore')
no_of_words = X_train[0].shape
array1=X_train[0]
no_of_train_messages = np.size(X_train,0)
#print("No of train messages:",no_of_train_messages)
#print("No of words in mail 1",no_of_words)
#print("no of words occurring from our vocabulary in mail 1",array1[array1==1].shape)


#calculating mail counts from training set
total_no_of_messages = X_train.shape[0]
total_no_of_spam_messages=X_train[y_train==1].shape[0]
total_no_of_ham_messages=X_train[y_train==0].shape[0]

#calculating probability from training set
probability_spam_messages = total_no_of_spam_messages/total_no_of_messages
probability_ham_messages = total_no_of_ham_messages/total_no_of_messages
print("probability P(y=1) of spam messages in training set:",probability_spam_messages)
print("probability P(y=0) of non spam messages in training set:",probability_ham_messages)



#separating data of spam and ham messages
spam_messages=X_train[y_train==1]
ham_messages=X_train[y_train==0]


total_spam_word_count=np.sum(spam_messages)
total_ham_word_count=np.sum(ham_messages)
count_of_words_in_ham=np.sum(ham_messages,0)        
count_of_words_in_spam=np.sum(spam_messages,0)  
print("total sum of spam words:",total_spam_word_count)
print("total sum of ham words:",total_ham_word_count)

#smoothing:adding 1 to the word count array to remove chance of 0 probability wherever we find the count of word as 0

for i in range(np.size(count_of_words_in_ham)):
     count_of_words_in_ham[i]=count_of_words_in_ham[i]+1
        
for i in range(np.size(count_of_words_in_spam)):
     count_of_words_in_spam[i]=count_of_words_in_spam[i]+1
        

#calculating probability for each word and storing it in array 


probability_spam_words_array=count_of_words_in_spam/total_spam_word_count
probability_ham_words_array=count_of_words_in_ham/total_ham_word_count
#print(probability_spam_words_array)
#print(probability_ham_words_array)

#getting the probability for word - secret and admirer
print("phi secret-spam:",probability_spam_words_array[vectorizer.vocabulary_.get('secret')])
print("phi secret-ham:",probability_ham_words_array[vectorizer.vocabulary_.get('secret')])
print("phi admirer-spam:",probability_spam_words_array[vectorizer.vocabulary_.get('admirer')])
print("phi admirer-ham:",probability_ham_words_array[vectorizer.vocabulary_.get('admirer')])

    
#classifying labels for test data

result=np.arange(np.size(y_test))
for i in range(np.size(y_test)): 
    likelihood_spam=0
    likelihood_ham=0
    posterior_spam_probability=0
    posterior_ham_probability=0
    message=X_test[i]
    k=1
    #print("data at index:",i,"data:",message)
    
    for k in range(np.size(message)):
        if message[k]==1:
            likelihood_spam+=np.log(probability_spam_words_array[k])
            likelihood_ham+=np.log(probability_ham_words_array[k])
    posterior_spam_probability=np.log(probability_spam_messages) +likelihood_spam
    posterior_ham_probability=np.log(probability_ham_messages) +likelihood_ham
    #print("posterior spam prob:",posterior_spam_probability)
    #print("posterior ham prob:",posterior_ham_probability)
    if(posterior_spam_probability>=posterior_ham_probability):
        result[i]=1
    else:
        result[i]=0


        
#printing result for first 50 classes
for i in range(50):
     print(result[i])       
        

        
#calculating total number of tests classfied correctly
Total_tests_classified_correctly=0
for i in range(np.size(y_test)):
    if(result[i]==y_test[i]):
        Total_tests_classified_correctly+=1
        
print("Total test examples classified correctly:",Total_tests_classified_correctly)
    

#calculating total number of tests classfied incorrectly
Total_tests_classified_incorrectly=0
for i in range(np.size(y_test)):
    if(result[i]!=y_test[i]):
        Total_tests_classified_incorrectly+=1
        
print("Total test examples classified correctly:",Total_tests_classified_incorrectly)
print("Percentage error on test examples:",(Total_tests_classified_incorrectly/np.size(y_test))*100)
        


        
        





probability P(y=1) of spam messages in training set: 0.13448193347690834
probability P(y=0) of non spam messages in training set: 0.8655180665230916
total sum of spam words: 8519
total sum of ham words: 24137
0
1
0
0
0
0
1
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
1
0
0
0
1
0
0
1
0
0
0
0
0
0
0
0
0
1
Total test examples classified correctly: 1368
Total test examples classified correctly: 25
Percentage error on test examples: 1.7946877243359656
phi secret-spam: 0.0010564620260593966
phi secret-ham: 0.00012429050834817913
phi admirer-spam: 0.0010564620260593966
phi admirer-ham: 4.143016944939305e-05
